In [ ]:
import paho.mqtt.client as PahoMQTT
import time
from raspClient import raspClient
from dataset import SignalGenerator
import tensorflow as tf
import numpy as np
import pathlib
import os
import datetime
from datetime import timezone
import json
from collections import Counter

In [ ]:
# DATASET PREPARATION -----------------------------------------
seed = 42
tf.random.set_seed(seed)
np.random.seed(seed)

# download dataset
data_dir = pathlib.Path('data/mini_speech_commands')
if not data_dir.exists():
  tf.keras.utils.get_file(
      'mini_speech_commands.zip',
      origin="http://storage.googleapis.com/download.tensorflow.org/data/mini_speech_commands.zip",
      extract=True,
      cache_dir='.', cache_subdir='data')

# lista di test
test_list=[]
file=open("./kws_test_split.txt")
for line in file:
  test_list.append('.'+line[1:-1])

# lista di labels - da replicare in EX1
labels = open('labels.txt').readlines()[0].split() 
print(labels)

MFCC_OPTIONS = {
    'frame_length': 640, 
    'frame_step': 320, 
    'mfcc': True,
    'lower_frequency': 20, 
    'upper_frequency': 4000, 
    'num_mel_bins': 40,
    'num_coefficients': 10
}

# make test dataset
generator = SignalGenerator(labels, 16000, **MFCC_OPTIONS)     
test_ds = generator.make_dataset(test_list, False)

# COOPERATIVE CLIENT (RASP)----------------------------------------------------------------
timestamp = int(datetime.datetime.now(timezone.utc).timestamp())
device_name = 'rasp'

subscribe_topics, publish_topic = ['/group11IOTHW3/model/1', 
                                   '/group11IOTHW3/model/2',
                                   '/group11IOTHW3/model/3',
                                   '/group11IOTHW3/model/4'], '/group11IOTHW3/prep_sample'

rasp = raspClient('raspClient', subscribe_topics, publish_topic, 4)
rasp.start()

for i, (mfcc, label) in enumerate(test_ds):
    senml_msg = {
        "bn" : device_name,
        "bt" : timestamp,
        "e" :[{"n":"audio", "u":"/", "t":0, "vd":mfcc.numpy().tolist(), 'idx': i}]
        }
    senml_msg = json.dumps(senml_msg)
    rasp.myPublish(publish_topic, senml_msg, i, label.numpy())
    
    time.sleep(0.1) # tune this parameter

In [ ]:
from collections import Counter
import numpy as np
a = [1,2,3,4,3,4,4,5,5,1,2,2,2]
counter = Counter(a)
elements = list(counter.items())
elements.sort(key= lambda x:x[1], reverse=True)
elements[0][0]

In [ ]:
a = dict()
a['a']